In [1]:
from keras.optimizers import Adam
from keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten, Dropout
from keras.layers import BatchNormalization, AveragePooling2D, concatenate
from keras.layers import Input, concatenate
from keras.layers import Add
from keras.layers import GlobalAveragePooling2D
from keras.models import Model

Using TensorFlow backend.


In [2]:
def Conv2d_BN(x, nb_filter, kernel_size, padding='same', strides=(1,1), name=None):
    if name is not None:
        bn_name = name +'_bn'
        conv_name = name + '_conv'
    else:
        bn_name = None
        conv_name = None
        
    
    x = Conv2D(nb_filter, kernel_size, padding=padding, strides=strides, activation='relu', name=conv_name)(x)
    x = BatchNormalization(axis=3, name=bn_name)(x)
    return x


In [3]:
def Residual_Block(input_model, nb_filter, kernel_size, strides=(1,1), with_conv_shortcut=False):
    x = Conv2d_BN(input_model, nb_filter=nb_filter, kernel_size=kernel_size, strides=strides, padding='same')
    x = Conv2d_BN(x, nb_filter=nb_filter, kernel_size=kernel_size, padding='same')
    
    if with_conv_shortcut:
        shorcut = Conv2d_BN(input_model, nb_filter, nb_filter, strides=strides, kernel_size=kernel_size)
        x = Add()([x, shortcut])
        return x
    else:
        x = Add()([x, input_model])
        return x

In [ ]:
def ResNet34(width, height, depth, classes):
    Img = Input(shape=(width, height, depth))
    
    x = Conv2d_BN(Img,64,(7,7), strides=(2,2), padding='same')
    x = MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='same')(x)
    
    #Residual conv2_x output 56x56x64
    x = Residual_Block(x, nb_filter=64, kernel_size=(3,3))
    x = Residual_Block()